In [23]:
import csv
import os
import itertools as itt
import classes as vp
import plotly.graph_objects as go
import plot
import mathHelper as mh

In [2]:
def loadSessions(file = 'sessions.csv', csv_data_dir = './csv_data'):
    #load session data form sessions.csv
    sessions = [] #list of all the sessions
    with open(file) as sessions_csv:
        sessionno = 0
        csv_reader = csv.reader(sessions_csv, delimiter=',')
        for l in csv_reader:
            sessions.append(vp.Session(sessionno, l[1], l[2], csv_data_dir + '/' + l[3]))
            sessionno += 1
    return sessions   

In [3]:
#todo: need output
def loadFactions(file = 'factions.csv', period='50'):
    factions_count = dict()
    with open(file) as factions_csv:
        csv_reader = csv.reader(factions_csv, delimiter=',')
        l = next(csv_reader)
        periodIndex = None
        for i,item in enumerate(l):
            if item == period:
                periodIndex = i
                break
        if periodIndex == None:
            #could not fined period
            return None
            
        l = next(csv_reader)
        while True:
            try:
                factions_count[l[0]] = int(l[periodIndex])
                l = next(csv_reader)
            except StopIteration:
                break
        return factions_count

In [4]:
def parseMems(r, session):
    #members = dict()
    posIdMap = []
    line = next(r)
    start = 0
    end = 0
    for i, item in enumerate(line):
        if item == 'CouncillorId':
            start = i+1
            break
    
    line = next(r)
    
    #enumerate ID
    for i, item in enumerate(line[start:]):
        if item == '':
            end = i + start
            break
            
        posIdMap.append(item)
        if not item in members:
            members[item] = vp.Member(item)
        (members[item].ismem).append(session.idno)

    ''' 
    #enumerate bioid
    line = next(r)    
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].bioid = item
    '''
    
    #enumerate name
    line = next(r)
    for i,item in enumerate(line[start:end]):
        #print(i), print(item)
        members[posIdMap[i]].name = item
        
    #skip line (this line only shows members are part of national council)
    next(r)
    #enumerate faction membership
    line = next(r)
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].faction = item 
        
    #enumerate canton
    line = next(r)
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].canton = item
        
    #enumerate birthdata
    line = next(r)
    for i,item in enumerate(line[start:end]):
        members[posIdMap[i]].birthdate = item

    

In [5]:
#parses all the sessions for the votes
def parseVotes(r, s):
    l = next(r)
    voters = []
    #votes = []
    voteno = 0
    start = 0
    while l[0] != 'VoteDate':
        l = next(r)
        start += 1
    
    #voters contains all the voters in a session
    endVotes = 0
    for i, item in enumerate(l[12:]):
        if item != 'Decision':
            voters.append(item)
        else:
            endVotes = i + 12
            break
    #print(len(voters))
    
    g = set()
    l = next(r)
    
    
    #need to handle:
    vote_none = '' #member was not yet or is no longer a national coucil member
    vote_yes = 'Ja'
    vote_no = 'Nein'
    vote_novote = ['Entschuldigt', 'Hat nicht teilgenommen']
    vote_abstain = 'Enthaltung'
    vote_pres = 'Der Präsident stimmt nicht'
    decision_set = {'yes', 'no'}
    
    
    while True:
        try:
            #store vote meta data
            v = vp.Vote(voteno)
            v.date = l[0]
            v.affairId = l[4]
            v.title = l[5]
            v.session = s
            v.sessionId = s.idno
            #store how each member voted by adding the members bioid to the yes/no/abstain/noset set
            for i, item in enumerate(l[12:endVotes]):
                if item == vote_none:
                    pass
                elif item == vote_yes:
                    v.yes.add(voters[i])
                elif item == vote_no:
                    v.no.add(voters[i])
                elif item == vote_abstain:
                    v.abstain.add(voters[i])
                elif item in vote_novote:
                    v.novote.add(voters[i])
                elif item == vote_pres:
                    pass
                else:
                    print('err vote: ' + item)
            
            #store decision data as stored in file and compare with calculated decision value
            v.decision = l[endVotes].lower()
            
            if v.decision != v.getDecision() or (not v.decision in decision_set):
                v.decision = 'err: \"' + v.decision + '\"' 
            votes.append(v)
            voteno += 1
            l = next(r)
        except StopIteration:
            break

In [6]:
#simple impirical voting power
#count how often a party in pivotal
#uses votes
def impVotingPower1(memList):
    pivotal = dict()
    for f in memList:
        count = 0
        for v in votes:
            if v.isPivotal(memList[f]):
                count += 1
        pivotal[f] = count #/ len(votes)
    return pivotal

In [7]:
# find the wining coelition for all the votes
# needs member dict
def allWinningC(votes):
    coelitions = dict()
    
    for i in range(1, len(vp.party_names) + 1):
        for p in itt.combinations(vp.party_names, i):
            key = list(p)
            key.sort(key=lambda x: vp.party_order[x])
            coelitions[', '.join(key)] = 0
    
    
    for i, v in enumerate(votes):
        try:
            coelitions[', '.join(winningC(v))] += 1
        except KeyError:
            print(v)
            print(i)
        
    return coelitions
    
    
#returns winning coelition for a vote
def winningC(vote):
    profiles = dict()
    for f in vp.party_names:
        profiles[f] = vp.VoteProfile()
    
    for mem in vote.yes:
        profiles[vp.faction_names[members[mem].faction]].yes += 1
        
    for mem in vote.no:
        profiles[vp.faction_names[members[mem].faction]].no += 1
        
    for mem in vote.abstain:
        profiles[vp.faction_names[members[mem].faction]].abstain += 1
        
    for mem in vote.novote:
        profiles[vp.faction_names[members[mem].faction]].novote += 1
        
    out = list()
    
    for p in profiles:
        if profiles[p].partyVote(2) == vote.decision:
            out.append(p)
    
    out.sort(key=lambda x: vp.party_order[x])
    return out
        
    

In [8]:
def parseFiles(sessions, toParse = {}): 
    #parse all the files
    #for testing: "filesToParse" can limit the parsed files (everything if empty)
    for s in sessions:
        if s.idno in toParse or len(toParse) == 0:
            with open(s.path) as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=',')
                print('-', end='')
                #print(str(key) + '. reading ... ' + csv_file.name)
                #parseFile(csv_reader)
                members = parseMems(csv_reader, s)
                parseVotes(csv_reader, s)
                csv_file.close

In [9]:
def genFactions():
    #list of members for each faction
    factions = dict()
    for mem in members:
        f = members[mem].faction
        if f in factions:
            factions[f].append(members[mem].bioid)
        else:
            factions[f] = [members[mem].bioid]
    return factions

In [10]:
def genFactionSize():
    #dict with size of each faction
    factions_size = dict()
    #factions_size['total'] = 0
    for f in factions:
        factions_size[f] = len(factions[f])
        #factions_size['total'] += len(factions[f])
        
    return factions_size

In [11]:
members = dict()
votes = []
memno = []

In [12]:
sessions = loadSessions()
parseFiles(sessions, toParse={})
factions_count_49 = loadFactions(period='49')
factions_count_50 = loadFactions(period='50')
factions = genFactions()
factions_size = genFactionSize()

---------------------------------------

In [13]:
allWinningC(votes)

{'None': 0,
 'SVP': 3,
 'GSP': 0,
 'GLP': 0,
 'SP': 0,
 'BDP': 0,
 'CVP': 0,
 'FDP': 1,
 'SVP, None': 0,
 'GSP, None': 0,
 'GLP, None': 0,
 'SP, None': 0,
 'BDP, None': 0,
 'CVP, None': 0,
 'FDP, None': 0,
 'GSP, SVP': 2,
 'GLP, SVP': 1,
 'SP, SVP': 2,
 'BDP, SVP': 2,
 'CVP, SVP': 43,
 'FDP, SVP': 171,
 'GSP, GLP': 1,
 'SP, GSP': 5,
 'GSP, BDP': 0,
 'GSP, CVP': 0,
 'GSP, FDP': 0,
 'SP, GLP': 0,
 'GLP, BDP': 0,
 'GLP, CVP': 0,
 'GLP, FDP': 0,
 'SP, BDP': 0,
 'SP, CVP': 0,
 'SP, FDP': 0,
 'CVP, BDP': 0,
 'BDP, FDP': 0,
 'CVP, FDP': 0,
 'GSP, SVP, None': 0,
 'GLP, SVP, None': 0,
 'SP, SVP, None': 4,
 'BDP, SVP, None': 0,
 'CVP, SVP, None': 2,
 'FDP, SVP, None': 4,
 'GSP, GLP, None': 0,
 'SP, GSP, None': 1,
 'GSP, BDP, None': 0,
 'GSP, CVP, None': 0,
 'GSP, FDP, None': 0,
 'SP, GLP, None': 0,
 'GLP, BDP, None': 0,
 'GLP, CVP, None': 0,
 'GLP, FDP, None': 0,
 'SP, BDP, None': 0,
 'SP, CVP, None': 0,
 'SP, FDP, None': 0,
 'CVP, BDP, None': 0,
 'BDP, FDP, None': 0,
 'CVP, FDP, None': 0,
 'GSP

In [15]:
#displays a histogram of all winning coelitions
def coelitions(simpleName = False, normalize=False):
    l = list(filter(lambda x: x[1] != 0, allWinningC(votes).items()))
    l.sort(key=lambda x: x[1], reverse=True)
    
    x = []
    y = []
    
    for i in l:
        if simpleName and (i[0] in coelition_names):
            x.append(coelition_names[i[0]])
        else:
            x.append(i[0])
        y.append(i[1])
    
    if normalize:
        #normalize data
        mh.norm(y)
    
    
    fig = go.Figure([go.Bar(x=x, y=y)])
    fig.show()

#simpler names for coelitions
coelition_names = {'SP, GSP, GLP, CVP, BDP, FDP': 'against SVP',
                  'CPV, BDP, FDP, SVP': 'center right',
                  'GLP, CVP, BDP, FDP, SVP' : 'against left',
                  'SP, GSP, GLP, CVP, BDP, FDP, SVP': 'unanimous',
                  'FDP, SVP': 'right',
                  'CVP, FDP, SVP': 'right and CVP',
                  'BDP, FDP, SVP': 'right and BDP',
                  'SP, GSP, GLP, BDP, FDP': 'center left'}
coelitions(normalize=True)

In [25]:
def showVoteDist(votes):
    #histogramm of how many members votes yes, no, abstain per vote
    present = []
    absent = []
    for v in votes:
        s = len(v.yes) + len(v.no) + len(v.abstain)
        present.append(s)
        absent.append(len(v.novote))
    plot.hist(present)
    plot.hist(absent)
showVoteDist(votes)

In [51]:
def memberPartisipation(votes):
    partisipation = dict()
    mems = []
    present = []
    absent = []
    
    def update(key, i, d=partisipation):
        
        key = vp.faction_names[members[key].faction]
        
        if not key in d:
            d[key] = [0,0,0,0]
        else:
            d[key][i] += 1
    
    def presents(x):
        return sum(x[0:3]) / sum(x)
    
    
    for v in votes:
        for mem in v.yes:
            update(mem, 0)
        for mem in v.no:
            update(mem, 1)
        for mem in v.abstain:
            update(mem, 2)
        for mem in v.novote:
            update(mem, 3)  
    
    
    for key in partisipation:
        mems.append(key)
        present.append(presents(partisipation[key]))
        
    fig = go.Figure([go.Bar(x=mems, y=present)])
    fig.show()
    #return partisipation
memberPartisipation(votes)

In [ ]:
#counts the members of each faction per session
factions_size_session = dict()
for mem in members:
    f = members[mem].faction
    for s in members[mem].ismem:
        if s in factions:
            if f in factions[s]:
                factions[s][f] += 1
            else:
                factions[s][f] = 1
        else:
            factions[s] = dict()
            factions[s][f] = 1
  


In [ ]:
vp.votingPower(factions_size)

In [ ]:
impVotingPower1(factions)

In [ ]:
pivotal = impVotingPower1(factions)

In [ ]:
pivotal

In [ ]:
def groupedBar_Dict(dicts, names, log=False):
    animals=['giraffes', 'orangutans', 'monkeys']
    
    data = []
    
    for k, d in enumerate(dicts):
        l = []
        for i in d:
            l.append(d[i] * 100)
        
        #for i,item in enumerate(l):
         #   l[i] /= sum(l)
        #norm(l)
        data.append(go.Bar(name=names[k], x=list(d.keys()), y=l, marker_color=plot.faction_color))

    fig = go.Figure(data=data)
    # Change the bar mode
    if log:
        fig.update_layout(barmode='group', yaxis_type='log')
    else:
        fig.update_layout(barmode='group')
    fig.show()
    
groupedBar_Dict([vp.PBI_votingPower(factions_count_49), vp.PBI_votingPower(factions_count_50)], ['49','50'])

In [65]:
m = []
m.append(['a', 3])
m.append(['b',5])

In [68]:
m

[['a', 3], ['b', 5]]